This notebook is created as a reference for SeqMetrics tests.
Since the SeqMetrics contain functions which are present in various other libraries, putting them as dependency while running automatic tests will
make testing quite difficult. Since it will make tests depends upon
a lot of libraries, some of which are no more maintained. Therefore,
in this notebook, we have installed those libraries and called those functions.
Some of the tests in SeqMetrics are compared against outputs of the functions
called in this notebook.


In [1]:
!pip install hydroeval

In [2]:
!pip install NeuralHydrology

In [3]:
!pip install HydroErr

In [4]:
!pip install SkillMetrics

In [5]:
!pip install RegscorePy

In [6]:
!pip install sktime

In [7]:
import datetime

import numpy as np

import xarray as xr

import pandas as pd

import hydroeval
from hydroeval import evaluator

import neuralhydrology

import HydroErr

import skill_metrics

import RegscorePy

import sktime

print(datetime.datetime.now().strftime("%d %B %Y %H:%M:%S"))
print('xarray version: ', xr.__version__)
print('np version: ', np.__version__)
print('pd version: ', pd.__version__)
print('hydroeval version: ', hydroeval.__version__)
print('sktime: ', sktime.__version__)

09 May 2024 13:02:31
xarray version:  2023.7.0
np version:  1.25.2
pd version:  2.0.3
hydroeval version:  0.1.0
sktime:  0.29.0


In [8]:
random_state = np.random.RandomState(seed=313)

t11 = random_state.random(100)
p11 = random_state.random(100)

t11.sum(), p11.sum()

(50.99521786026271, 57.734075639710824)

In [9]:
"""
The helper functions in this cell are copied from https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9
"""
EPSILON = 1e-10


def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted


def _percentage_error(actual: np.ndarray, predicted: np.ndarray):
    """
    Percentage error
    Note: result is NOT multiplied by 100
    """
    return _error(actual, predicted) / (actual + EPSILON)


def _naive_forecasting(actual: np.ndarray, seasonality: int = 1):
    """ Naive forecasting method which just repeats previous samples """
    return actual[:-seasonality]


def _relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1
        else:
            seasonality = benchmark
        return _error(actual[seasonality:], predicted[seasonality:]) /\
               (_error(actual[seasonality:], _naive_forecasting(actual, seasonality)) + EPSILON)

    return _error(actual, predicted) / (_error(actual, benchmark) + EPSILON)

def _bounded_relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Bounded Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1
        else:
            seasonality = benchmark

        abs_err = np.abs(_error(actual[seasonality:], predicted[seasonality:]))
        abs_err_bench = np.abs(_error(actual[seasonality:], _naive_forecasting(actual, seasonality)))
    else:
        abs_err = np.abs(_error(actual, predicted))
        abs_err_bench = np.abs(_error(actual, benchmark))

    return abs_err / (abs_err + abs_err_bench + EPSILON)


def _geometric_mean(a, axis=0, dtype=None):
    """ Geometric mean """
    if not isinstance(a, np.ndarray):  # if not an ndarray object attempt to convert it
        log_a = np.log(np.array(a, dtype=dtype))
    elif dtype:  # Must change the default dtype allowing array type
        if isinstance(a, np.ma.MaskedArray):
            log_a = np.log(np.ma.asarray(a, dtype=dtype))
        else:
            log_a = np.log(np.asarray(a, dtype=dtype))
    else:
        log_a = np.log(a)
    return np.exp(log_a.mean(axis=axis))


def mae(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(_error(actual, predicted)))

def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    return np.mean(np.square(_error(actual, predicted)))

def mape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Absolute Percentage Error
    Properties:
        + Easy to interpret
        + Scale independent
        - Biased, not symmetric
        - Undefined when actual[t] == 0
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.abs(_percentage_error(actual, predicted)))



### mda

In [10]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L259

def mda(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Directional Accuracy """
    return np.mean((np.sign(actual[1:] - actual[:-1]) == np.sign(predicted[1:] - actual[:-1])).astype(int))


mda(t11, p11)

0.696969696969697

### mbrae

In [11]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L248

def mbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Bounded Relative Absolute Error """
    return np.mean(_bounded_relative_error(actual, predicted, benchmark))

mbrae(t11, p11)

0.46659593775205116

### umbrae

In [12]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L253
def umbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Unscaled Mean Bounded Relative Absolute Error """
    __mbrae = mbrae(actual, predicted, benchmark)
    return __mbrae / (1 - __mbrae)

umbrae(t11, p11)

0.8747513766311694

### gmrae

In [13]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L243

def gmrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Geometric Mean Relative Absolute Error """
    return _geometric_mean(np.abs(_relative_error(actual, predicted, benchmark)))

gmrae(t11, p11)

0.79938390310645

### mdrae

In [14]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L238

def mdrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Median Relative Absolute Error """
    return np.median(np.abs(_relative_error(actual, predicted, benchmark)))

mdrae(t11, p11)

0.9086455067666214

### mrae

In [15]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L233

def mrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Absolute Error """
    return np.mean(np.abs(_relative_error(actual, predicted, benchmark)))

mrae(t11, p11)

2.5711621568850163

### rae

In [16]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L228

def rae(actual: np.ndarray, predicted: np.ndarray):
    """ Relative Absolute Error (aka Approximation Error) """
    return np.sum(np.abs(actual - predicted)) / (np.sum(np.abs(actual - np.mean(actual))) + EPSILON)

rae(t11, p11)

1.3287945409387383

### mre

In [17]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L223

def mre(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Error """
    return np.mean(_relative_error(actual, predicted, benchmark))

mre(t11, p11)

0.5101139564068491

### rrse

In [18]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L223

def rrse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Relative Squared Error """
    return np.sqrt(np.sum(np.square(actual - predicted)) / np.sum(np.square(actual - np.mean(actual))))

rrse(t11, p11)

1.4381836641228662

### inrse

In [19]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L213

def inrse(actual: np.ndarray, predicted: np.ndarray):
    """ Integral Normalized Root Squared Error """
    return np.sqrt(np.sum(np.square(_error(actual, predicted))) / np.sum(np.square(actual - np.mean(actual))))

inrse(t11, p11)

1.4381836641228662

### rmsse

In [20]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L207

def rmsse(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """ Root Mean Squared Scaled Error """
    q = np.abs(_error(actual, predicted)) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))
    return np.sqrt(np.mean(np.square(q)))

rmsse(t11,p11)

1.2234619716320643

### me

In [21]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L86

def me(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Error """
    return np.mean(_error(actual, predicted))

me(t11, p11)

-0.06738857779448111

### rmse

In [22]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L76

def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))

rmse(t11, p11)

0.40289487147518754

### nrmse

In [23]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L81

def nrmse(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Root Mean Squared Error """
    return rmse(actual, predicted) / (actual.max() - actual.min())

nrmse(t11, p11)

0.4081874143525102

### mpe

In [24]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L112

def mpe(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Percentage Error """
    return np.mean(_percentage_error(actual, predicted))

mpe(t11, p11)

-42.20843064537674

### gmae

In [25]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L102

def gmae(actual: np.ndarray, predicted: np.ndarray):
    """ Geometric Mean Absolute Error """
    return _geometric_mean(np.abs(_error(actual, predicted)))

gmae(t11, p11)

0.19423992928498718

### mdape

In [26]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L132

def mdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(np.abs(_percentage_error(actual, predicted)))

mdape(t11, p11)

0.513246349701827

### smape

In [27]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L141

def smape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Mean Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))

smape(t11, p11)

0.7028826489278417

### smdape

In [28]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L150

def smdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))

smdape(t11, p11)

0.5999121382638821

### maape

In [29]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L159

def maape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Arctangent Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.arctan(np.abs((actual - predicted) / (actual + EPSILON))))

maape(t11, p11)

0.5828454707567975

### mase

In [30]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L168

def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """
    Mean Absolute Scaled Error
    Baseline (benchmark) is computed with naive forecasting (shifted by @seasonality)
    """
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))

mase(t11, p11)

0.9609397361653512

### std_ae
renamed as norm_ae in seqmetrics


In [31]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L177

def std_ae(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Error """
    __mae = mae(actual, predicted)
    return np.sqrt(np.sum(np.square(_error(actual, predicted) - __mae))/(len(actual) - 1))

std_ae(t11, p11)

0.5551510970200795

### std_ape

renamed as norm_ape in seqmetrics

In [32]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L183

def std_ape(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Percentage Error """
    __mape = mape(actual, predicted)
    return np.sqrt(np.sum(np.square(_percentage_error(actual, predicted) - __mape))/(len(actual) - 1))

std_ape(t11, p11)

404.0632278812062

### rmspe

In [33]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L189

def rmspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Mean Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.mean(np.square(_percentage_error(actual, predicted))))

rmspe(t11, p11)

395.25283254969173

### rmdspe

In [34]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L198

def rmdspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Median Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.median(np.square(_percentage_error(actual, predicted))))

rmdspe(t11, p11)

0.5133222853161394

### NSE

In [35]:
from hydroeval import nse

# Calculate NSE
nse_value = evaluator(nse, p11, t11)
print('NSE:', nse_value[0])


NSE: -1.0683722517498735


### KGE

In [36]:
from hydroeval import kge as kge

# Calculate KGE
kge_value = evaluator(kge, p11, t11)
print('KGE:', kge_value[0])

KGE: [0.00897063]


### alpha_nse

In [37]:

from neuralhydrology.evaluation.metrics import alpha_nse

alpha_nse(xr.DataArray(t11), xr.DataArray(p11))

1.0235046034233621

### beta_nse

In [38]:
from neuralhydrology.evaluation.metrics import beta_nse

beta_nse(xr.DataArray(t11), xr.DataArray(p11))

0.2405519617999516

### nse_mod

In [39]:

from HydroErr import nse_mod
nse_mod(p11, t11)

-0.32879454094431804

### relative nse

In [40]:
from HydroErr import nse_rel

nse_rel(p11, t11)

-517670.8159599439

### nse_bound

In [41]:
from hydroeval import nse_c2m

nse_c2m(p11, t11)

-0.34818860428052284

### kge_bound

In [42]:

from hydroeval import kge_c2m

kge_c2m(p11, t11)

array([0.00450552])

### kge_mod

In [43]:

from HydroErr import kge_2012

kge_2012(p11, t11)

0.004612979178136856

### kge_np

In [44]:

from hydroeval import kgenp

evaluator(kgenp, p11, t11)

array([[-0.00671877],
       [ 0.00244824],
       [ 0.96982724],
       [ 1.13214686]])

### pbias

In [45]:

from hydroeval import pbias

evaluator(pbias, p11, t11)

array([-13.21468573])

In [46]:
pbias(p11, t11)

-13.21468573369753

In [47]:
from skill_metrics import bias_percent

bias_percent(p11, t11)

13.214685733697538

### bias

In [48]:
from skill_metrics import bias

In [49]:
bias(p11, t11)

0.06738857779448115

### gmae

In [50]:

# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L102

def gmae(actual: np.ndarray, predicted: np.ndarray):
    """ Geometric Mean Absolute Error """
    return _geometric_mean(np.abs(_error(actual, predicted)))

gmae(t11, p11)

0.19423992928498718

### mase

In [51]:

# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L168

def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """
    Mean Absolute Scaled Error
    Baseline (benchmark) is computed with naive forecasting (shifted by @seasonality)
    """
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))

mase(t11, p11)

0.9609397361653512

### fdc_fhv

In [52]:
from neuralhydrology.evaluation.metrics import fdc_fhv

In [53]:
fdc_fhv(xr.DataArray(t11), xr.DataArray(p11))

1.5933766138626262

### fdc_flv

In [54]:
from neuralhydrology.evaluation.metrics import fdc_flv

In [55]:
fdc_flv(xr.DataArray(t11), xr.DataArray(p11))

32.64817835760714

### maape

In [56]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L159

def maape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Arctangent Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.arctan(np.abs((actual - predicted) / (actual + EPSILON))))

maape(t11, p11)

0.5828454707567975

### lm_index

In [57]:
from HydroErr import lm_index

lm_index(p11, t11)

-0.32879454094431804

### kgenp_bound

In [58]:
from hydroeval import kgenp_c2m

In [59]:
evaluator(kgenp_c2m, p11, t11)

array([-0.00334814])

### kgeprime_c2m

In [60]:
from hydroeval import kgeprime_c2m

In [61]:
evaluator(kgeprime_c2m, p11, t11)

array([0.00231182])

### mle

In [62]:
from HydroErr import mle

In [63]:
mle(p11, t11)

0.0438958324374804

### modified agreement of index

In [64]:
from HydroErr import dmod

In [65]:
dmod(p11, t11)

0.36018092524466827

### mpe

In [66]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L112

def mpe(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Percentage Error """
    return np.mean(_percentage_error(actual, predicted))

In [67]:
mpe(t11, p11)

-42.20843064537674

### mrae

In [68]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L233

def mrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Absolute Error """
    return np.mean(np.abs(_relative_error(actual, predicted, benchmark)))

mrae(t11, p11)

2.5711621568850163

### nrmse

In [69]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L81

def nrmse(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Root Mean Squared Error """
    return rmse(actual, predicted) / (actual.max() - actual.min())

nrmse(t11, p11)

0.4081874143525102

### nrmse_ipercentile

In [70]:
from HydroErr import nrmse_iqr

nrmse_iqr(p11, t11)

0.8187123709758822

### nrmse_mean

In [71]:
from HydroErr import nrmse_mean

nrmse_mean(p11, t11)

0.790064026354788

### acc

In [72]:
from HydroErr import acc

acc(p11, t11)

0.0179208383645756

### ve

In [73]:
from HydroErr import ve

ve(p11, t11)

0.3794625949621073

### watt_m

In [74]:
from HydroErr import watt_m

watt_m(p11, t11)

0.017290316806567577

### skill score murphy

In [75]:
from skill_metrics import skill_score_murphy

skill_score_murphy(p11, t11)

-1.0476885292323743

### sid

In [76]:
from HydroErr import sid

sid(p11, t11)

43.71192101756139

### smdape

In [77]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L150

def smdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))

smdape(t11, p11)

0.5999121382638821

### smape

In [78]:
from HydroErr import smape2

smape2(p11, t11)

70.28826490215243

### sc

In [79]:
from HydroErr import sc

sc(p11, t11)

1.5526934811208075

### sa

In [80]:
from HydroErr import sa

sa(p11, t11)

0.6618474080345743

### rmspe

In [81]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L189

def rmspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Mean Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.mean(np.square(_percentage_error(actual, predicted))))

rmspe(t11, p11)

395.25283254969173

### relative agreement index

In [82]:
from HydroErr import drel

drel(p11, t11)

-139396.49261170527

### refined agreement index

In [83]:
from HydroErr import dr

dr(p11, t11)

0.335602729527841

### rmdspe

In [84]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L198

def rmdspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Median Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.median(np.square(_percentage_error(actual, predicted))))

rmdspe(t11, p11)

0.5133222853161394

### normalized ape

In [85]:
def mape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Absolute Percentage Error
    Properties:
        + Easy to interpret
        + Scale independent
        - Biased, not symmetric
        - Undefined when actual[t] == 0
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.abs(_percentage_error(actual, predicted)))

# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L183
def std_ape(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Percentage Error """
    __mape = mape(actual, predicted)
    return np.sqrt(np.sum(np.square(_percentage_error(actual, predicted) - __mape))/(len(actual) - 1))

std_ape(t11, p11)

404.0632278812062

### normalized ae

In [86]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L177

def std_ae(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Error """
    __mae = mae(actual, predicted)
    return np.sqrt(np.sum(np.square(_error(actual, predicted) - __mae))/(len(actual) - 1))

std_ae(t11, p11)

0.5551510970200795

### normalized euclid distance

In [87]:
from HydroErr import ned

ned(p11, t11)

7.338597737626875

### euclid distance

In [88]:
from HydroErr import ed

ed(p11, t11)

4.028948714751875

# relative rmse

In [89]:
# https://stackoverflow.com/a/41749834
# https://stackoverflow.com/q/69360839
# https://gist.github.com/AayushSameerShah/baeedbbb52fe808874d6debb7e786183


### mean variance

In [90]:
from HydroErr import mean_var

mean_var(p11, t11)

0.07449144510570738

### median error

In [91]:
from HydroErr import mde

mde(p11, t11)

0.0313854202641316

### mdape

In [92]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L132

def mdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(np.abs(_percentage_error(actual, predicted)))
mdape(t11, p11)

0.513246349701827

### Mielke Berry R

In [93]:
from HydroErr import mb_r

mb_r(p11, t11)

0.04444743269492335

### aic

In [94]:
from RegscorePy import aic

In [95]:
aic.aic(t11, p11, p=1)

-179.8159231784877

### bic

In [96]:
import collections

collections.Sequence = list   # because bic function uses collections.Sequence

from RegscorePy import bic

bic.bic(y=t11, y_pred=p11, p=1)

-177.2107529924996

### wmape

In [97]:
# https://stackoverflow.com/a/54833202/5982232

def wmape(actual, forecast):
    # Take a series (actual) and a dataframe (forecast) and calculate wmape
    # for each forecast. Output shape is (1, num_forecasts)

    # Convert to numpy arrays for broadasting
    forecast = np.array(forecast.values)
    actual=np.array(actual.values).reshape((-1, 1))

    # Make an array of mape (same shape as forecast)
    se_mape = abs(actual-forecast)/actual

    # Calculate sum of actual values
    ft_actual_sum = actual.sum(axis=0)

    # Multiply the actual values by the mape
    se_actual_prod_mape = actual * se_mape

    # Take the sum of the product of actual values and mape
    # Make sure to sum down the rows (1 for each column)
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum(axis=0)

    # Calculate the wmape for each forecast and return as a dictionary
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum
    return {f'Forecast_{i+1}_wmape': wmape for i, wmape in enumerate(ft_wmape_forecast)}


wmape(pd.Series(t11), pd.DataFrame(p11))

{'Forecast_1_wmape': 0.6205374050378927}

### irmse

In [98]:
from HydroErr import irmse

irmse(p11, t11)

0.9954807723243245

### CronbachAlpha

In [99]:
# https://stackoverflow.com/a/20799687

def CronbachAlpha(itemscores):
    itemscores = np.asarray(itemscores)
    itemvars = itemscores.var(axis=1, ddof=1)
    tscores = itemscores.sum(axis=0)
    nitems = len(itemscores)

    return nitems / (nitems-1.) * (1 - itemvars.sum() / tscores.var(ddof=1))


CronbachAlpha(np.stack([t11, p11]))

0.03555058748735895

### aitchison distance

In [100]:
# https://github.com/hallamlab/utilities/blob/master/SparCC/lib/distances.py#L47

def aitchison(x,y, center = 'mean'):
    lx = np.log(x)
    ly = np.log(y)
    if center == 'mean':     m = np.mean
    elif center == 'median': m = np.median
    clr_x = lx - m(lx)
    clr_y = ly - m(ly)
    d = ( sum((clr_x-clr_y)**2) )**0.5
    return d


aitchison(t11, p11)

16.326288844358846

### JS

In [101]:
# https://github.com/hallamlab/utilities/blob/master/SparCC/lib/distances.py#L19

def JS(x,y): #Jensen-shannon divergence
    import warnings
    warnings.filterwarnings("ignore", category = RuntimeWarning)
    d1 = x*np.log2(2*x/(x+y))
    d2 = y*np.log2(2*y/(x+y))
    d1[np.isnan(d1)] = 0
    d2[np.isnan(d2)] = 0
    d = 0.5*sum(d1+d2)
    return d


JS(t11, p11)

7.275875413762115

### Median Squared Error

In [102]:
from sktime.performance_metrics.forecasting import MedianSquaredError

mdse = MedianSquaredError()
mdse(t11, p11)


0.06731204476856545